In [ ]:
# Databricks notebook source
import mlflow
from mlflow.tracking import MlflowClient
import sys
import os

# ==================== CONFIGURATION ====================

# 💡 महत्वपूर्ण: यह Experiment Name, model_train.py में प्रयुक्त नाम से मेल खाना चाहिए।
EXPERIMENT_NAME = "/Shared/House_Price_Prediction_Delta_RF"

# ⚠️ UNITY CATALOG के लिए नामकरण (MANDATORY)
UC_CATALOG_NAME = "workspace"
UC_SCHEMA_NAME = "ml"
REGISTERED_MODEL_NAME = f"{UC_CATALOG_NAME}.{UC_SCHEMA_NAME}.house_price_model_uc" 

MODEL_ARTIFACT_PATH = "sklearn_rf_model"

# मॉडल पैरामीटर जिन्हें हम तुलना के लिए उपयोग करेंगे (model_train.py से 'best_' prefixes का उपयोग)
PARAM_KEYS = ['best_n_estimators', 'best_max_depth', 'best_min_samples_split', 'best_min_samples_leaf']

# मॉडल मेट्रिक (Metric) जिसे हम comparison के लिए उपयोग करेंगे (model_train.py से 'test_rmse')
METRIC_KEY = 'test_rmse' 
METRIC_TOLERANCE = 1e-6 # मेट्रिक की तुलना के लिए स्वीकार्य त्रुटि (tolerance)

# ==================== FUNCTIONS ====================

def normalize_param_value(value):
	"""
	पैरामीटर की value को normalize करता है ताकि comparison सही हो सके।
	MLflow कभी-कभी parameters को string के रूप में store करता है।
	"""
	if value is None:
		return None
	
	# String को appropriate type में convert करें
	str_value = str(value)
	
	# Check if it's a number
	try:
		# पहले integer try करें
		if '.' not in str_value and str_value.lstrip('-').isdigit():
			return int(str_value)
		else:
			return float(str_value)
	except (ValueError, TypeError):
		# यदि number नहीं है तो string ही return करें
		return str_value


def get_latest_run_info(client: MlflowClient):
	"""
	निर्दिष्ट एक्सपेरिमेंट में हाल ही में सफल हुए रन का ID, Parameters, और Metrics प्राप्त करता है।
	"""
	try:
		experiment = client.get_experiment_by_name(EXPERIMENT_NAME)
		if not experiment:
			print(f"❌ Error: Experiment '{EXPERIMENT_NAME}' not found.")
			return None, {}, {}

		# सभी रन को शुरू होने के समय के अनुसार क्रमबद्ध करें
		runs = client.search_runs(
			experiment_ids=[experiment.experiment_id],
			order_by=["start_time DESC"],
			max_results=1,
			# metrics और params को साथ में fetch करें
			filter_string="attributes.status = 'FINISHED'" 
		)
		
		if runs:
			latest_run = runs[0]
			run_id = latest_run.info.run_id
			
			# Parameters fetch करें (चारों 'best_' पैरामीटर्स)
			run_params = {}
			for key in PARAM_KEYS:
				if key in latest_run.data.params:
					param_value = latest_run.data.params[key]
					run_params[key] = normalize_param_value(param_value)
			
			# Metric fetch करें ('test_rmse')
			run_metrics = {}
			if METRIC_KEY in latest_run.data.metrics:
				run_metrics[METRIC_KEY] = latest_run.data.metrics[METRIC_KEY]
			
			print(f"✓ Latest Run ID found: {run_id}")
			print(f"  Params: {run_params}")
			print(f"  Metrics: {run_metrics}")
			
			return run_id, run_params, run_metrics
		else:
			print(f"⚠ Warning: No FINISHED runs found in experiment '{EXPERIMENT_NAME}'.")
			return None, {}, {}
			
	except Exception as e:
		print(f"❌ Error fetching latest run info: {e}")
		return None, {}, {}


def get_run_info_from_version(client: MlflowClient, model_name: str, version_number: str):
	"""
	Model version से associated run के Parameters और Metrics निकालता है।
	"""
	try:
		version_details = client.get_model_version(model_name, version_number)
		
		# Version से run_id निकालें
		run_id = version_details.run_id
		
		if not run_id:
			print(f"⚠ Warning: No run_id found for version {version_number}")
			return {}, {}
		
		# Run से parameters और metrics निकालें
		run = client.get_run(run_id)
		
		# Parameters
		run_params = {}
		for key in PARAM_KEYS:
			if key in run.data.params:
				param_value = run.data.params[key]
				run_params[key] = normalize_param_value(param_value)
		
		# Metrics
		run_metrics = {}
		if METRIC_KEY in run.data.metrics:
			run_metrics[METRIC_KEY] = run.data.metrics[METRIC_KEY]

		return run_params, run_metrics
		
	except Exception as e:
		print(f"⚠ Warning: Could not fetch info for version {version_number}: {e}")
		return {}, {}


def check_existing_version(client: MlflowClient, current_params: dict, current_metrics: dict):
	"""
	जाँच करता है कि क्या समान पैरामीटर और मेट्रिक वाला मॉडल संस्करण पहले से ही पंजीकृत है।
	"""
	
	current_metric_value = current_metrics.get(METRIC_KEY)
	
	if current_metric_value is None:
		print(f"❌ Error: Required metric '{METRIC_KEY}' not found in current run. Cannot perform metric comparison.")
		# यदि मेट्रिक नहीं मिलती है, तो केवल पैरामीटर तुलना पर वापस आएं
		print("ℹ️ Falling back to parameters-only comparison.")
		current_metric_value = -1 # Sentinel value, won't match existing versions easily

	
	print(f"\n🔍 Checking for existing versions with parameters: {current_params} and metric ({METRIC_KEY}): {current_metric_value:.6f}")
	
	try:
		# Unity Catalog में सभी model versions खोजें
		filter_string = f"name = '{REGISTERED_MODEL_NAME}'"
		versions = client.search_model_versions(filter_string=filter_string)
		
		if not versions:
			print(f"ℹ️ Model '{REGISTERED_MODEL_NAME}' does not exist in Unity Catalog. Proceeding with registration.")
			return None
		
		print(f"📋 Found {len(versions)} existing version(s). Checking parameters and metrics...")
		
		for version in versions:
			try:
				# Version से associated run के parameters और metrics निकालें
				version_params, version_metrics = get_run_info_from_version(
					client, 
					REGISTERED_MODEL_NAME, 
					version.version
				)
				
				version_metric_value = version_metrics.get(METRIC_KEY)

				print(f"   Version {version.version}: Params={version_params}, {METRIC_KEY}={version_metric_value}")
				
				# 1. Parameters की तुलना करें (Exact Match)
				params_match = all(
					key in version_params and version_params[key] == current_params.get(key)
					for key in PARAM_KEYS
				)
				
				# 2. Metrics की तुलना करें (Match within Tolerance)
				metrics_match = False
				if version_metric_value is not None and current_metric_value != -1: 
					# abs(a - b) < tolerance
					if abs(current_metric_value - version_metric_value) <= METRIC_TOLERANCE:
						metrics_match = True

				
				if params_match and metrics_match:
					print("\n" + "#" * 70)
					print(f"⏭️  DUPLICATE DETECTED: Version {version.version} already exists")
					print(f"   Parameters AND Metric ({METRIC_KEY}) match within tolerance.")
					print(f"   Existing {METRIC_KEY}: {version_metric_value:.6f} | Current {METRIC_KEY}: {current_metric_value:.6f}")
					print(f"   ✅ Skipping registration to avoid unnecessary versioning.")
					print("#" * 70 + "\n")
					return version
				
			except Exception as e:
				print(f"   ⚠ Error checking version {version.version}: {e}")
				continue
				
		print("✓ No duplicate (by Params + Metric) found. Proceeding with new registration.\n")
		return None 	# कोई डुप्लीकेट नहीं मिला
		
	except Exception as e:
		error_msg = str(e).upper()
		if "RESOURCE_DOES_NOT_EXIST" in error_msg or "PERMISSION_DENIED" in error_msg:
			print(f"ℹ️ Model '{REGISTERED_MODEL_NAME}' not found or Permission Denied in UC.")
			print(f" 	 Proceeding with registration as first version.\n")
			return None
		
		# अन्य errors के लिए
		print(f"⚠ Warning: Error checking existing versions in UC: {e}")
		print(f" 	 Proceeding with registration (caution advised).\n")
		return None


def register_model_for_serving(client: MlflowClient, run_id: str, run_params: dict, run_metrics: dict, model_name: str, artifact_path: str):
	"""
	MLflow Run से मॉडल को Unity Catalog Model Registry में पंजीकृत करता है।
	पहले डुप्लीकेट चेक करता है।
	"""
	
	# 1. डुप्लीकेट जाँच
	existing_version = check_existing_version(client, run_params, run_metrics)
	if existing_version:
		print(f"🎯 Using existing model version: {existing_version.name} v{existing_version.version}")
		return existing_version

	# 2. पंजीकरण
	model_uri = f"runs:/{run_id}/{artifact_path}"
	print(f"⏳ Attempting to register NEW model version from URI: {model_uri}")
	
	try:
		# MLflow स्वतः ही UC का उपयोग करेगा
		model_version = mlflow.register_model(
			model_uri=model_uri, 
			name=model_name
		)
		
		print("\n" + "=" * 60)
		print("✅ UC MODEL REGISTRATION SUCCESSFUL!")
		print(f"नाम: {model_version.name}")
		print(f"संस्करण (Version): {model_version.version}")
		print(f"स्टेज (Stage): {model_version.current_stage}")
		print(f"Parameters: {run_params}")
		print(f"Metrics ({METRIC_KEY}): {run_metrics.get(METRIC_KEY)}")
		print("=" * 60 + "\n")
		
		return model_version
		
	except Exception as e:
		print(f"❌ UC मॉडल पंजीकरण विफल हुआ! त्रुटि: {e}")
		print("सुनिश्चित करें कि आपके पास Unity Catalog में मॉडल बनाने की अनुमति है।")
		sys.exit(1)


if __name__ == "__main__":
	
	print("\n" + "=" * 60)
	print("🚀 MLFLOW MODEL REGISTRATION WITH DUPLICATE CHECK (Params + Metric)")
	print("=" * 60 + "\n")
	
	# UC के लिए पुराने Workspace Registry कॉन्फ़िगरेशन को हटा दें
	try:
		if "MLFLOW_REGISTRY_URI" in os.environ:
			del os.environ["MLFLOW_REGISTRY_URI"]
			print("✓ Removed MLFLOW_REGISTRY_URI environment variable for UC registration.")
	except Exception as e:
		print(f"⚠ Warning: Could not remove MLFLOW_REGISTRY_URI: {e}")

	# Mlflow क्लाइंट को इनिशियलाइज़ करें
	client = MlflowClient()

	# 1. रन ID, पैरामीटर और मेट्रिक्स को स्वचालित रूप से प्राप्त करें
	# यह फ़ंक्शन अब 'best_n_estimators', 'test_rmse' आदि को फेच करेगा।
	RUN_ID, CURRENT_RUN_PARAMS, CURRENT_RUN_METRICS = get_latest_run_info(client)
	
	if RUN_ID:
		# डुप्लीकेट चेक और पंजीकरण (Registration)
		registered_version = register_model_for_serving(
			client, 
			RUN_ID, 
			CURRENT_RUN_PARAMS,
			CURRENT_RUN_METRICS,
			REGISTERED_MODEL_NAME, 
			MODEL_ARTIFACT_PATH
		)
		
		if registered_version:
			print(f"\n✅ Final Result:")
			print(f" 	Model: {registered_version.name}")
			print(f" 	Version: {registered_version.version}")
			print(f"\n💡 अगला कदम: इस UC मॉडल संस्करण का उपयोग करके Serving Endpoint बनाएँ।")
	else:
		print("❌ Registration aborted: Could not retrieve a valid Run ID.")
